In [ ]:
Latam-challenge de Franco Talamo (ftalamo)
Instrucciones
1. Descargue el código fuente desde https://github.com/ftalamo/latam-challenge/
2. Crear una carpeta en el directory src con en nombre de data
3. abre el sitio web de google cloud https://console.cloud.google.com/
4. configura un nuevo proyecto.
    4.1 coloca como dominio autorizado google.com
    4.2 coloca tu correo de google como email del desarrollador
5. Habilita la API de Google Drive 
6. Configura la pantalla de consentimiento
    6.1  En tipo de usuario seleccionar externo
    6.2  Colocar un nombre a la app
    6.3  Agregar el correo
    6.4 En dominio autorizado colocar google.com
    6.5 En correo de desarrollador agregar el email
7.- Ir a credenciales
    7.1 Seleccionar cliente Oauth
    7.2 Seleccionar aplicación web
    7.3 En URI colocar http://localhost:8080
    7.4 EN URI de REDIRECCIONAMIENTO colocar http://localhost:8080/
8. Una vez creadas las credenciales descargar el archivo json y renombrar a client_secrets.json
9. Guardamos el archivo json en src/
10. una vez configurado nuestro env ejecutar *pip install -r requirements.txt*
11. ejecutar el archivo *src/modules/getdata.py*
    11.1 nos abrira una ventana de autenticación de google y aceptamos 
12. crear en el directorio raiz el archivo settings.yaml
    12.1 el contenido del archivo debe ser:
        12.1.1 client_config_backend: settings
               client_config:
                   client_id:[sacar el dato del archivo client_secrets.json]
                   client_secret: [sacar el dato del archivo client_secrets.json]
               save_credentials: True
               save_credentials_backend: file
               save_credentials_file: credentials_module.json
13. ejecutar el paso 11 nuevamente
    13.1 nos abrira una ventana de autenticaión de google y aceptamos
    13.2 se creará el archivo credentials_module.json en el directorio src/
14. una vez generado el archivo credentials_module.json no es necesario ejecutar la autenticación nuevamente ya que se hara de forma automatica
15. ejecutar el archivo *src/challenge.py*
16. ver resultados y mediciones de tiempo en el archivo performance.txt que se creara en la raiz del proyecto



'''clase que ocupa pydrive2 para conectarse al API de google drive '''
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive


class Oauth:

    def __init__(self, creds_module): #recibe el archivo de credenciales
        self.creds_module = creds_module

    def login(self):
        gauth = GoogleAuth()
        gauth.LoadCredentialsFile(self.creds_module) #carga las credenciales 
        if gauth.access_token_expired:
            gauth.Refresh() #si están expiradas o incorrectas refresca el archivo
            gauth.SaveCredentialsFile(self.creds_module)
        else:
            gauth.Authorize() # autoriza la conexión

        return GoogleDrive(gauth) #retorna un objeto de Autorización a endpoint de drive
        




In [ ]:
''' getdata ocupa el modulo Oauth para conectarse a Drive '''
from Oauth import Oauth


def getdata(file_id, credentials, destination): # recibe el id del archivo a descargar, las credenciales y el path de donde guardara el archivo

    service = Oauth(credentials) # crea una nueva instancia de Oauth 
    service = service.login()  # ejecuta el login (si se siguieron los pasos de la guia de instrucciones no solicitara permisos)

    # Descargar el archivo zip
    file = service.CreateFile({'id': file_id}) #Verifica el archivo
    file_name = file['title'] #guarda el nombre
    file.GetContentFile(destination+file_name) #guarda el archivo
    return file_name #retorna el nombre del archivo para futuro uso


In [ ]:
''' función unzip. Descomprime el archivo zip descargado '''
import zipfile


def unzip_file(zip_file, extract_to): #recibe el nombre del archivo que devuelve get data y donde se guardará
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to) #hace la extraccción
        return zip_ref.infolist() #devuelve una lista con la metadata del archivo


In [ ]:
'''Readjson lee el archivo de datos en formato json '''
import json


def read_json(file_path): #recibe como parametro el path de ubicación del archivo
    try:
        tweets = []
        with open(file_path, 'r', encoding='utf-8') as file: #abre el archivo
            for line in file: #lee linea a linea 
                try:
                    tweet = json.loads(line)
                    tweets.append(tweet)
                except json.JSONDecodeError as e:
                    print(f"Error al decodificar la línea JSON: {str(e)}")
        return tweets
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo {file_path}.") # en caso de no encontrar el archivo arroja error
        return None #guaarda el archivo en memoria


In [ ]:
'''format date toma la fecha en en formato timestamp y lo convierte al solicitado en el challenge'''
from datetime import datetime


def format_date(date):
    try:
        # Convertir la fecha a formato datetime
        fecha_datetime = datetime.strptime(date, "%Y-%m-%dT%H:%M:%S+00:00")

        # Obtener solo la parte de la fecha en formato deseado
        fecha_formateada = fecha_datetime.strftime("%Y,%m,%d")

        return fecha_formateada
    except ValueError:
        raise ValueError("Formato de fecha inválido")


In [ ]:
''' measure_performance es el modulo encargado de medir los tiempo de ejecución y el gasto de memoria de cada ejecución y guardarlo
en el archivo performance.txt '''

import psutil
import timeit
from datetime import datetime
from functools import wraps


def measure_performance(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = timeit.default_timer()
        result = func(*args, **kwargs)
        elapsed_time = format(timeit.default_timer() - start_time,".4f")
        execution_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # Obtener el uso de memoria del proceso
        process = psutil.Process()
        memory_usage = format(process.memory_info().rss / (1024 * 1024),".4f")  # Convertir a MB

        # Guardar los resultados en un archivo
        with open("../performance.txt", "a", encoding="utf-8") as file:
            file.write('**********************************************\n\n')
            file.write(f"Fecha y hora de ejecución: {execution_time}\n")
            file.write(f"Función: {func.__name__}, Tiempo: {elapsed_time} segundos, Uso de memoria: {memory_usage} Mb\n Resultado: {result}\n\n")
        return result
    return wrapper

In [ ]:
'''challenge.py es el archivo principal del proyecto y el que se debe ejecutar para correr todos los scripts 
de la app'''

import os
from modules.getdata import getdata #importa de la carpeta modulos todos los modulos necesarios
from modules.unzip import unzip_file
from q1_memory import q1_memory
from q1_time import q1_time #importa los archivo q# del challenge
from q2_memory import q2_memory
from q2_time import q2_time
from q3_memory import q3_memory
from q3_time import q3_time


if __name__ == "__main__":
    """" define los datos para enviar a google drive y descargar la data """
    credentials = 'credentials_module.json'
    data_dir = 'data/'
    file_id = '1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis'

    '''verifica que exista el archivo antes de ir a buscarlo en drive'''
    datafile = 'data/farmers-protest-tweets-2021-2-4.json'
    if os.path.exists(datafile):
        '''si existe el archivo copia el nombre'''
        jdfile_name = datafile
    else:
        '''descarga el archivo y lo descomprime desde drive'''
        try:
            datafile = getdata(file_id, credentials, data_dir) # usa el modulo get data para conectarse a drive
            jsondatafile = unzip_file(data_dir + datafile, data_dir) #guarda en esta variable la metadata del json
            jdfile_name = data_dir + jsondatafile[0].filename
        except Exception as e:
            print(f"error: {e}")
            print('Por favor ve al link https://drive.usercontent.google.com/download?id=1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis&export=download&authuser=0 y descarga de forma manual el archivo')

    q1_memory_optimize = q1_memory(jdfile_name) # ejecuta el script 
    print(f"q1_memory result: {q1_memory_optimize}") # imprime el resultado (mismo que se guarda en el archivo performace)
    q1_time_optimize = q1_time(jdfile_name) # ejecuta el script 
    print(f"q1_time result: {q1_time_optimize}") # imprime el resultado (mismo que se guarda en el archivo performace)
    q2_memory_optimize = q2_memory(jdfile_name) # ejecuta el script 
    print(f"q2_memory result: {q2_memory_optimize}") # imprime el resultado (mismo que se guarda en el archivo performace)
    q2_time_optimize = q2_time(jdfile_name) # ejecuta el script 
    print(f"q2_time result: {q2_time_optimize}") # imprime el resultado (mismo que se guarda en el archivo performace)
    q3_memory_optimize = q3_memory(jdfile_name) # ejecuta el script 
    print(f"q3_memory result: {q3_memory_optimize}") # imprime el resultado (mismo que se guarda en el archivo performace)
    q3_time_optimize = q3_time(jdfile_name) # ejecuta el script 
    print(f"q3_time result: {q3_time_optimize}") # imprime el resultado (mismo que se guarda en el archivo performace)






In [ ]:
'''la principal diferencia entre los q#_memory y los q$_time es que los Q#_time usan una función
previa donde genera un buffer con la información del archivo json y desde ese buffer abre un pool
de conexiones para leer y escribir por chunks (definidos por el usuario), lo que debería mejorar la performance 
siempre y cuando se cuente con procesadores multihilos'''
